In [2]:
import os

print('teslat')

teslat


In [16]:
from random import choice, random, sample
import numpy as np
import numexpr as ne

# TODOs
#   - better expressions (expression tree make it easier to get ground truth by evaluating)
#   - store in a local db


def gen_arithmetic_expressions(numbers, operators, num_exprs=10):
    """Generate an arithmetic expression"""
    paren_prob = 0.25
    operators_str = ['{{}} {op} {{}}'.format(op=op) for op in operators]
    expressions = numbers[:]
    
    for _ in range(150):
        expression = choice(operators_str).format(choice(expressions), choice(expressions))
        if random() < paren_prob:
            # wrap the expression in parentheses
            expression = '({})'.format(expression)
        expressions.append(expression)
        
    return sample(expressions, num_exprs)


class ArithmeticQuestion:
    def __init__(self, left, cmp, right):
        self.left = left
        self.cmp = cmp
        self.right = right
    
    def evaluate(self):
        expression = ''.join((self.left + self.cmp + self.right).split())
        evaluated = ne.evaluate(expression)
        return evaluated
    
    def __repr__(self):
        return '{left} {cmp} {right} ?'.format(left=self.left, cmp=self.cmp, right=self.right)
        

def generate_questions(n):
    """Generate synthetic data"""
    operators = ['*', '-', '+', '**']    # todo - add expr
    numbers = list(map(str, range(100)))
    
    left_expressions = gen_arithmetic_expressions(numbers, operators, num_exprs=n)
    right_expressions = sample(numbers, n)
    
    comparators = ['<', '>', '==']
    generated_questions = [ArithmeticQuestion(left=left, cmp=sample(comparators, 1)[0], right=right) 
                           for (left, right) in list(zip(left_expressions, right_expressions))]
    return generated_questions


def generate_questions_and_labels(n):
    return [(q, generate_label(q)) for q in generate_questions(n)]


def generate_label(x):
    if isinstance(x, ArithmeticQuestion):
        return x.evaluate()

questions_and_labels = generate_questions_and_labels(n=10)
print(questions_and_labels)

[(53 + 53 > 35 ?, array(True, dtype=bool)), (53 == 50 ?, array(False, dtype=bool)), ((68 + 90) > 96 ?, array(True, dtype=bool)), (5 - 76 == 73 ?, array(False, dtype=bool)), (5 == 90 ?, array(False, dtype=bool)), ((23 * 47) == 82 ?, array(False, dtype=bool)), (11 < 39 ?, array(True, dtype=bool)), (42 + 10 < 37 ?, array(False, dtype=bool)), (5 * 2 > 33 ?, array(False, dtype=bool)), (54 > 4 ?, array(True, dtype=bool))]
